# Imports

In [ ]:
from pathlib import Path

import pandas as pd

import constants.column_names.dashboard as col_dashboard
import constants.column_names.silver as col_silver
from constants.paths import SILVER_YIELD_PATH

In [ ]:
SCENARII_DIR = Path("../models/baseline_xgboost_model/artifacts")
SCENARIO_126_PATH = SCENARII_DIR / "scenario_126_predictions.csv"
SCENARIO_245_PATH = SCENARII_DIR / "scenario_245_predictions.csv"
SCENARIO_585_PATH = SCENARII_DIR / "scenario_585_predictions.csv"

# Financial impact

To measure financial impact, we compare for a fixed area the price of production at average yield with the price of production by selecting departments to optimize yield (to do this, we assume we drop the $20$ departments with the lowest yield, and that we can relocate their production to the $69$ remaining departments).

The price of production is given by the formula:
$$
Price = Yield \times Area \times Barley\ Price 
$$

We take the barley price to be 120 $/ton which is the globa price of barley in January 2026 according to the [FRED](https://fred.stlouisfed.org/series/PBARLUSDM) database.

In [ ]:
BARLEY_PRICE = 120  # $/ton

In [ ]:
historical_yield_data = pd.read_parquet(SILVER_YIELD_PATH)

In [ ]:
avg_area_per_dep = historical_yield_data.groupby(col_silver.SILVER_NOM_DEP).agg(
    {col_silver.SILVER_AREA: "mean"}
)
avg_area_cultivated_per_dep = avg_area_per_dep.mean()

average_area_cultivated_per_year = (
    historical_yield_data.groupby(col_silver.SILVER_YEAR)
    .agg({col_silver.SILVER_AREA: "sum"})
    .mean()
)

number_of_deps_needed = (
    average_area_cultivated_per_year / avg_area_cultivated_per_dep
).values[0]

In [ ]:
scenario_126 = pd.read_csv(SCENARIO_126_PATH)
scenario_245 = pd.read_csv(SCENARIO_245_PATH)
scenario_585 = pd.read_csv(SCENARIO_585_PATH)

best_yield_126 = (
    scenario_126.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .nlargest(columns=col_dashboard.DASHBOARD_YIELD, n=70)
    .mean()
)
best_yield_245 = (
    scenario_245.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .nlargest(columns=col_dashboard.DASHBOARD_YIELD, n=70)
    .mean()
)
best_yield_585 = (
    scenario_585.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .nlargest(columns=col_dashboard.DASHBOARD_YIELD, n=70)
    .mean()
)

print(f"Best yield for scenario 126: {best_yield_126.values[0]:.2f} tons/ha")
print(f"Best yield for scenario 245: {best_yield_245.values[0]:.2f} tons/ha")
print(f"Best yield for scenario 585: {best_yield_585.values[0]:.2f} tons/ha")


average_yield_scenario_126 = (
    scenario_126.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .mean()
)
average_yield_scenario_245 = (
    scenario_245.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .mean()
)
average_yield_scenario_585 = (
    scenario_585.groupby(col_dashboard.DASHBOARD_DEPARTMENT_NAME)
    .agg({col_dashboard.DASHBOARD_YIELD: "mean"})
    .mean()
)

print(
    f"\nAverage yield for scenario 126: {average_yield_scenario_126.values[0]:.2f} "
    "tons/ha"
)
print(
    f"Average yield for scenario 245: {average_yield_scenario_245.values[0]:.2f} "
    "tons/ha"
)
print(
    f"Average yield for scenario 585: {average_yield_scenario_585.values[0]:.2f} "
    "tons/ha"
)

In [ ]:
financial_impact_scenario_126 = (
    (best_yield_126.values[0] - average_yield_scenario_126.values[0])
    * avg_area_cultivated_per_dep.values[0]
    * BARLEY_PRICE
) * number_of_deps_needed

financial_impact_scenario_245 = (
    (best_yield_245.values[0] - average_yield_scenario_245.values[0])
    * avg_area_cultivated_per_dep.values[0]
    * BARLEY_PRICE
) * number_of_deps_needed

financial_impact_scenario_585 = (
    (best_yield_585.values[0] - average_yield_scenario_585.values[0])
    * avg_area_cultivated_per_dep.values[0]
    * BARLEY_PRICE
) * number_of_deps_needed

print(f"\nFinancial impact for scenario 126: ${financial_impact_scenario_126:,.2f}")
print(f"Financial impact for scenario 245: ${financial_impact_scenario_245:,.2f}")
print(f"Financial impact for scenario 585: ${financial_impact_scenario_585:,.2f}")

average_financial_impact = (
    financial_impact_scenario_126
    + financial_impact_scenario_245
    + financial_impact_scenario_585
) / 3

print(f"\nAverage financial impact across scenarios: ${average_financial_impact:,.2f}")